In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm,multivariate_normal

In [2]:
###### FLEMING'S FUNCTION MODIFIED
def compute_meta_conf_serialdependence(xp, a, sigma_act, sigma_conf, rho):
    
    dhat = np.array([-1, 1])
    mu_x_xp_dhat = np.zeros((2, len(xp)))
    var_x_xp_dhat = np.zeros(len(xp))
    rho_vec = np.full(len(xp), rho)
    sigA_vec = np.full(len(xp), sigma_act)
    sigP_vec = np.full(len(xp), sigma_conf)
    
    Tol = 10e-4

    for dhati in range(2):
        dhat_vec = np.full(len(xp), dhat[dhati])
        
        mu_x_xp_dhat[dhati, :] = dhat_vec + (sigA_vec / sigP_vec) * rho_vec * (xp - dhat_vec)
        var_x_xp_dhat = (1 - rho_vec**2) * sigA_vec**2
        
        if a == 1:
            p_a_dhat_xp = 1 - norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        else:
            p_a_dhat_xp = norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        
        lik_d = norm.pdf(xp, dhat_vec, sigP_vec)
        
        if dhati == 0:
            p_a_dhat_xp_full = p_a_dhat_xp
            lik_d_full = lik_d
        else:
            p_a_dhat_xp_full = np.vstack((p_a_dhat_xp_full, p_a_dhat_xp))
            lik_d_full = np.vstack((lik_d_full, lik_d))
    
    # manage probability
    p_a_dhat_xp_full = np.clip(p_a_dhat_xp_full, Tol, None)
    lik_d_full = np.clip(lik_d_full, Tol, None)
    
    lik_d_full = lik_d_full / np.sum(lik_d_full, axis=0, keepdims=True)
    p_dhat_xp_a = p_a_dhat_xp_full * lik_d_full
    p_dhat_xp_a = p_dhat_xp_a / np.sum(p_dhat_xp_a, axis=0, keepdims=True)
    
    # Conf = p(a=d)
    if a == 1:
        conf = p_dhat_xp_a[1, :]
    else:
        conf = p_dhat_xp_a[0, :]
    
    return conf

In [3]:
# Supongamos que tenemos dos modelos, uno con 3 parámetros y otro con 5 parámetros.
# Definimos las funciones model1_run y model2_run

def model1_run(d, a, rho, sigmaConf, theta):
    # Aquí iría tu lógica específica para model1 con 3 parámetros
    sigmaAct = 1
    bigSigma = np.array([[sigmaAct**2, rho * sigmaAct * sigmaConf], [rho * sigmaAct * sigmaConf, sigmaConf**2]])

    N = len(d)
    xa = np.empty(N)
    xp = np.empty(N)
    secondOrder_mean_cor = np.empty(N)

    for i in range(N):
        current_theta = theta
        r = multivariate_normal.rvs(mean=[d[i] * current_theta, d[i] * current_theta], cov=bigSigma)
        
        xa[i] = r[0]
        xp[i] = r[1]
        
        flip_a = a[i]

        secondOrder_mean_cor[i] = compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]

    secondOrder_mean_cor_adj = secondOrder_mean_cor[1:]

    return secondOrder_mean_cor_adj

def model2_run(d, a, rho, sigmaConf, theta, alpha, beta):
    # Aquí iría tu lógica específica para model2 con 5 parámetros
    sigmaAct = 1
    bigSigma = np.array([[sigmaAct**2, rho * sigmaAct * sigmaConf], [rho * sigmaAct * sigmaConf, sigmaConf**2]])

    N = len(d)
    xa = np.empty(N)
    xp = np.empty(N)
    secondOrder_mean_cor_serialDependence =  np.full( N, 111.0)
    first_trial = True 

    for i in range(N):
        current_theta = theta
        r = multivariate_normal.rvs(mean=[d[i] * current_theta, d[i] * current_theta], cov=bigSigma)
        
        xa[i] = r[0]
        xp[i] = r[1]
        
        flip_a = a[i]
        
        if first_trial == False:
            p_serial_dependence = np.random.beta(alpha, beta, 1)[0]
            secondOrder_mean_cor_serialDependence[i] = (
                p_serial_dependence * secondOrder_mean_cor_serialDependence[i-1] + 
                (1-p_serial_dependence) * compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
            )
        else:
            secondOrder_mean_cor_serialDependence[i] = compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
    
        first_trial = False

    # drop first value
    secondOrder_mean_cor_serialDependence_adj = secondOrder_mean_cor_serialDependence[1:]

    return secondOrder_mean_cor_serialDependence_adj


# Función negativa de log-likelihood para model1 (con 3 parámetros)
def negative_log_likelihood_model1(params, data):
    rho, sigmaConf, theta = params
    all_secondOrder_mean_cor = []

    for n_p in data['Subj_idx'].unique():
        df_vp_participant = data[data['Subj_idx'] == n_p]
        d = df_vp_participant['Stimulus_transformed'].values
        a = df_vp_participant['Response_transformed'].values
        secondOrder_mean_cor = model1_run(d, a, rho, sigmaConf, theta)
        all_secondOrder_mean_cor.extend(secondOrder_mean_cor)

    all_secondOrder_mean_cor = np.array(all_secondOrder_mean_cor)
    grouped = df_vp.groupby('Subj_idx')
    df_vp_cleaned = pd.concat([group.iloc[1:] for _, group in grouped]).reset_index(drop=True)
    conf_data = df_vp_cleaned['Confidence_0to1'].values

    epsilon = 1e-10
    all_secondOrder_mean_cor = np.clip(all_secondOrder_mean_cor, epsilon, 1 - epsilon)
    log_likelihood = np.sum(norm.logpdf(conf_data, all_secondOrder_mean_cor))

    return -log_likelihood

# Función negativa de log-likelihood para model2 (con 5 parámetros)
def negative_log_likelihood_model2(params, data):
    rho, sigmaConf, theta, alpha, beta = params
    all_secondOrder_mean_cor = []

    for n_p in data['Subj_idx'].unique():
        df_vp_participant = data[data['Subj_idx'] == n_p]
        d = df_vp_participant['Stimulus_transformed'].values
        a = df_vp_participant['Response_transformed'].values
        secondOrder_mean_cor = model2_run(d, a, rho, sigmaConf, theta, alpha, beta)
        all_secondOrder_mean_cor.extend(secondOrder_mean_cor)

    all_secondOrder_mean_cor = np.array(all_secondOrder_mean_cor)
    grouped = df_vp.groupby('Subj_idx')
    df_vp_cleaned = pd.concat([group.iloc[1:] for _, group in grouped]).reset_index(drop=True)
    conf_data = df_vp_cleaned['Confidence_0to1'].values

    epsilon = 1e-10
    all_secondOrder_mean_cor = np.clip(all_secondOrder_mean_cor, epsilon, 1 - epsilon)
    log_likelihood = np.sum(norm.logpdf(conf_data, all_secondOrder_mean_cor))

    return -log_likelihood

# Definimos una función para calcular AIC y BIC
def calculate_aic_bic(log_likelihood, num_params, num_observations):
    aic = 2 * num_params - 2 * log_likelihood
    bic = np.log(num_observations) * num_params - 2 * log_likelihood
    return aic, bic

# Cargar datos
df = pd.read_csv('data_Mazancieux_2018.csv')
#suj = [1,2,3]
#df_vp = df[(df['Task'] == 'VP') & (df['Subj_idx'].isin(suj))].copy()
df_vp = df[df['Task'] == 'EM'].copy()
df_vp.loc[:, 'Stimulus_transformed'] = df_vp['Stimulus'].replace({1: -1, 2: 1})
df_vp.loc[:, 'Response_transformed'] = df_vp['Response'].replace({1: 0, 2: 1})
df_vp['Confidence_0to1'] = df_vp['Confidence'] / 10

# Número de observaciones
num_observations = len(df_vp)

# Optimización de los parámetros para model1
initial_guess_model1 = [0.5, 1.0, 0.5]  # Supongamos que estos son los parámetros iniciales para model1
bounds_model1 = [(0.0001, 0.9999), (0.1, 2), (0.1, 0.9)]
result_model1 = minimize(negative_log_likelihood_model1, initial_guess_model1, args=(df_vp,), bounds=bounds_model1, method='Powell')
params_model1 = result_model1.x
log_likelihood_model1 = -result_model1.fun

# Optimización de los parámetros para model2
initial_guess_model2 = [0.5, 1.0, 0.5, 3, 7]  # Supongamos que estos son los parámetros iniciales para model2
bounds_model2 = [(0.0001, 0.9999), (0.1, 2), (0.1, 0.9), (1, 20), (1, 20)]
result_model2 = minimize(negative_log_likelihood_model2, initial_guess_model2, args=(df_vp,), bounds=bounds_model2, method='Powell')
params_model2 = result_model2.x
log_likelihood_model2 = -result_model2.fun

# Calcular AIC y BIC para cada modelo
num_params_model1 = len(params_model1)
num_params_model2 = len(params_model2)

aic_model1, bic_model1 = calculate_aic_bic(log_likelihood_model1, num_params_model1, num_observations)
aic_model2, bic_model2 = calculate_aic_bic(log_likelihood_model2, num_params_model2, num_observations)

# Comparar los resultados
results = [
    {'model': 'model1', 'log_likelihood': log_likelihood_model1, 'aic': aic_model1, 'bic': bic_model1},
    {'model': 'model2', 'log_likelihood': log_likelihood_model2, 'aic': aic_model2, 'bic': bic_model2}
]

# Convertir resultados a DataFrame para mejor visualización
results_df = pd.DataFrame(results)
print(results_df)

    model  log_likelihood           aic           bic
0  model1    -7013.658000  14033.316000  14053.978130
1  model2    -6944.571776  13899.143553  13933.580435
